## Streaming Real Time Options Data with ThetaData

In [ ]:
!pip install thetadata

In [ ]:
import datetime as dt
from thetadata import ThetaClient, OptionRight, StreamMsgType, StreamResponseType

In [ ]:
def callback(msg):
    # Handle received messages
    if msg.type == StreamMsgType.TRADE:
        print("----------------------------------------------------")
        print(f"Contract: {msg.contract.to_string()}")
        print(f"Trade: {msg.trade.to_string()}")
        print(f"Last quote at time of trade: {msg.quote.to_string()}")

In [ ]:
def stream_all_trades():
    # Initialize the ThetaClient with your credentials
    client = ThetaClient(username="YOUR_EMAIL", passwd="YOUR_PASSWORD")

    # Connect to the data stream
    client.connect_stream(callback)

    # Request full trade stream for options
    req_id = client.req_full_trade_stream_opt()

    # Verify subscription
    if client.verify(req_id) != StreamResponseType.SUBSCRIBED:
        raise Exception("Unable to stream.")

Start streaming all trades

In [ ]:
stream_all_trades()

Define parameters for a specific option contract

In [ ]:
ticker = "SPY"
expiration_date = dt.date(2024, 12, 22)
strike = 474

In [ ]:
def stream_contract():
    # Initialize the ThetaClient with your credentials
    client = ThetaClient(username="YOUR_EMAIL", passwd="YOUR_PASSWORD")

    # Connect to the data stream
    client.connect_stream(callback)

    # Request trade stream for a specific option contract
    req_id = client.req_trade_stream_opt(
        ticker, expiration_date, strike, OptionRight.CALL
    )

    # Verify subscription
    if client.verify(req_id) != StreamResponseType.SUBSCRIBED:
        raise Exception("Unable to stream.")

Start streaming trades for a specific contract

In [ ]:
stream_contract()

**Jason Strimpel** is the founder of <a href='https://pyquantnews.com/'>PyQuant News</a> and co-founder of <a href='https://www.tradeblotter.io/'>Trade Blotter</a>. His career in algorithmic trading spans 20+ years. He previously traded for a Chicago-based hedge fund, was a risk manager at JPMorgan, and managed production risk technology for an energy derivatives trading firm in London. In Singapore, he served as APAC CIO for an agricultural trading firm and built the data science team for a global metals trading firm. Jason holds degrees in Finance and Economics and a Master's in Quantitative Finance from the Illinois Institute of Technology. His career spans America, Europe, and Asia. He shares his expertise through the <a href='https://pyquantnews.com/subscribe-to-the-pyquant-newsletter/'>PyQuant Newsletter</a>, social media, and has taught over 1,000+ algorithmic trading with Python in his popular course **<a href='https://gettingstartedwithpythonforquantfinance.com/'>Getting Started With Python for Quant Finance</a>**. All code is for educational purposes only. Nothing provided here is financial advise. Use at your own risk.

## Streaming Real Time Options Data with ThetaData

In [ ]:
import datetime as dt
from thetadata import (
    Quote,
    StreamMsg,
    ThetaClient,
    OptionRight,
    StreamMsgType,
    StreamResponseType,
)

Initialize global variables for last quotes and price

In [ ]:
last_call_quote = Quote()
last_put_quote = Quote()
price = 0

In [ ]:
def callback_straddle(msg):
    # Handle received messages
    if msg.type != StreamMsgType.QUOTE:
        return

    global price

    # Update the last call or put quote based on the message
    if msg.contract.isCall:
        last_call_quote.copy_from(msg.quote)
    else:
        last_put_quote.copy_from(msg.quote)

    # Calculate straddle bid, ask, and mid prices
    straddle_bid = round(last_call_quote.bid_price + last_put_quote.bid_price, 2)
    straddle_ask = round(last_call_quote.ask_price + last_put_quote.ask_price, 2)
    straddle_mid = round((straddle_bid + straddle_ask) / 2, 2)

    # Get the timestamp from the message
    time_stamp = thetadata.client.ms_to_time(msg.quote.ms_of_day)

    # Print the straddle prices if they have changed
    if price != straddle_mid:
        print(
            f"time: {time_stamp} bid: {straddle_bid} mid: {straddle_mid} ask: {straddle_ask}"
        )
        price = straddle_mid

In [ ]:
def stream_straddle():
    # Initialize the ThetaClient with your credentials
    client = ThetaClient(username="YOUR_EMAIL", passwd="YOUR_PASSWORD")
    
    # Connect to the data stream
    client.connect_stream(callback_straddle)
    
    # Request quote streams for call and put options
    req_id_call = client.req_quote_stream_opt(
        "SPY", dt.date(2024, 3, 28), 475, OptionRight.CALL
    )
    req_id_put = client.req_quote_stream_opt(
        "SPY", dt.date(2024, 3, 28), 475, OptionRight.PUT
    )
    
    # Verify the subscriptions
    if (
        client.verify(req_id_call) != StreamResponseType.SUBSCRIBED
        or client.verify(req_id_put) != StreamResponseType.SUBSCRIBED
    ):
        raise Exception("Unable to stream.")

Start streaming straddle quotes

In [ ]:
stream_straddle()

**Jason Strimpel** is the founder of <a href='https://pyquantnews.com/'>PyQuant News</a> and co-founder of <a href='https://www.tradeblotter.io/'>Trade Blotter</a>. His career in algorithmic trading spans 20+ years. He previously traded for a Chicago-based hedge fund, was a risk manager at JPMorgan, and managed production risk technology for an energy derivatives trading firm in London. In Singapore, he served as APAC CIO for an agricultural trading firm and built the data science team for a global metals trading firm. Jason holds degrees in Finance and Economics and a Master's in Quantitative Finance from the Illinois Institute of Technology. His career spans America, Europe, and Asia. He shares his expertise through the <a href='https://pyquantnews.com/subscribe-to-the-pyquant-newsletter/'>PyQuant Newsletter</a>, social media, and has taught over 1,000+ algorithmic trading with Python in his popular course **<a href='https://gettingstartedwithpythonforquantfinance.com/'>Getting Started With Python for Quant Finance</a>**. All code is for educational purposes only. Nothing provided here is financial advise. Use at your own risk.

## Streaming Real Time Options Data with ThetaData

In [ ]:
import datetime as dt
from thetadata import (
    Quote,
    StreamMsg,
    ThetaClient,
    OptionRight,
    StreamMsgType,
    StreamResponseType,
)

Define the option contract parameters for the Iron Condor

In [ ]:
ticker = "SPY"
expiration_date = dt.date(2024, 3, 28)
long_put_strike = 460
short_put_strike = 465
short_call_strike = 480
long_call_strike = 485

Initialize global variables for option quotes and price

In [ ]:
long_put = Quote()
short_put = Quote()
short_call = Quote()
long_call = Quote()
price = 0

In [ ]:
def callback_iron_condor(msg):
    # Handle received messages
    if msg.type != StreamMsgType.QUOTE:
        return
    
    global price
    
    # Update the relevant quote based on the message
    if not msg.contract.isCall and msg.contract.strike == long_put_strike:
        long_put.copy_from(msg.quote)
    elif not msg.contract.isCall and msg.contract.strike == short_put_strike:
        short_put.copy_from(msg.quote)
    elif msg.contract.isCall and msg.contract.strike == short_call_strike:
        short_call.copy_from(msg.quote)
    elif msg.contract.isCall and msg.contract.strike == long_call_strike:
        long_call.copy_from(msg.quote)
    
    # Calculate Iron Condor bid, ask, and mid prices
    condor_bid = round(
        long_put.bid_price
        - short_put.bid_price
        + long_call.bid_price
        - short_call.bid_price,
        2,
    )
    condor_ask = round(
        long_put.ask_price
        - short_put.ask_price
        + long_call.ask_price
        - short_call.ask_price,
        2,
    )
    condor_mid = round((condor_ask + condor_bid) / 2, 2)

    # Get the timestamp from the message
    time_stamp = thetadata.client.ms_to_time(msg.quote.ms_of_day)
    
    # Print the Iron Condor prices if they have changed
    if price != condor_mid:
        print(
            f"time: {time_stamp} bid: {condor_bid} mid: {condor_mid} ask: {condor_ask}"
        )
        price = condor_mid

In [ ]:
def stream_iron_condor():
    # Initialize the ThetaClient with your credentials
    client = ThetaClient(username="YOUR_EMAIL", passwd="YOUR_PASSWORD")
    
    # Connect to the data stream
    client.connect_stream(callback_iron_condor)
    
    # Request quote streams for all four legs of the Iron Condor
    lp_id = client.req_quote_stream_opt(
        ticker, expiration_date, long_put_strike, OptionRight.PUT
    )
    sp_id = client.req_quote_stream_opt(
        ticker, expiration_date, short_put_strike, OptionRight.PUT
    )
    sc_id = client.req_quote_stream_opt(
        ticker, expiration_date, short_call_strike, OptionRight.CALL
    )
    lc_id = client.req_quote_stream_opt(
        ticker, expiration_date, long_call_strike, OptionRight.CALL
    )
    
    # Verify the subscriptions
    if (
        client.verify(lp_id) != StreamResponseType.SUBSCRIBED
        or client.verify(sp_id) != StreamResponseType.SUBSCRIBED
        or client.verify(sc_id) != StreamResponseType.SUBSCRIBED
        or client.verify(lc_id) != StreamResponseType.SUBSCRIBED
    ):
        raise Exception("Unable to stream.")

Start streaming Iron Condor quotes

In [ ]:
stream_iron_condor()

**Jason Strimpel** is the founder of <a href='https://pyquantnews.com/'>PyQuant News</a> and co-founder of <a href='https://www.tradeblotter.io/'>Trade Blotter</a>. His career in algorithmic trading spans 20+ years. He previously traded for a Chicago-based hedge fund, was a risk manager at JPMorgan, and managed production risk technology for an energy derivatives trading firm in London. In Singapore, he served as APAC CIO for an agricultural trading firm and built the data science team for a global metals trading firm. Jason holds degrees in Finance and Economics and a Master's in Quantitative Finance from the Illinois Institute of Technology. His career spans America, Europe, and Asia. He shares his expertise through the <a href='https://pyquantnews.com/subscribe-to-the-pyquant-newsletter/'>PyQuant Newsletter</a>, social media, and has taught over 1,000+ algorithmic trading with Python in his popular course **<a href='https://gettingstartedwithpythonforquantfinance.com/'>Getting Started With Python for Quant Finance</a>**. All code is for educational purposes only. Nothing provided here is financial advise. Use at your own risk.

## Using the ArcticDB DataFrame Database for Tick Storage

In [ ]:
!conda install -c conda-forge arcticdb -y

In [ ]:
from IPython.display import display, Markdown
import time
import pytz
import datetime as dt
import pandas as pd
import arcticdb as adb

In [ ]:
from thetadata import (
    ThetaClient,
    OptionRight,
    StreamMsg,
    StreamMsgType,
    StreamResponseType,
)

Initialize ArcticDB and create a library for trades

In [ ]:
arctic = adb.Arctic("lmdb://arcticdb_options")
lib = arctic.get_library("trades", create_if_missing=True)

Function to get the trade datetime

In [ ]:
def get_trade_datetime(today, ms_of_day):
    return today + dt.timedelta(milliseconds=ms_of_day)

Function to get the number of days to expiration

In [ ]:
def get_days_to_expiration(today, expiration):
    return (expiration - today).days

Callback function to handle received messages

In [ ]:
def callback(msg):
    today = dt.datetime.now(pytz.timezone("US/Eastern")).replace(
        hour=0, minute=0, second=0, microsecond=0
    )

    if msg.type == StreamMsgType.TRADE:
        trade_datetime = get_trade_datetime(today, msg.trade.ms_of_day)
        expiration = pd.to_datetime(msg.contract.exp).tz_localize("US/Eastern")
        days_to_expiration = get_days_to_expiration(today, expiration)
        symbol = msg.contract.root
        trade = {
            "root": symbol,
            "expiration": expiration,
            "days_to_expiration": days_to_expiration,
            "is_call": msg.contract.isCall,
            "strike": msg.contract.strike,
            "size": msg.trade.size,
            "trade_price": msg.trade.price,
            "exchange": str(msg.trade.exchange.value[1]),
            "bid_size": msg.quote.bid_size,
            "bid_price": msg.quote.bid_price,
            "ask_size": msg.quote.ask_size,
            "ask_price": msg.quote.ask_price,
        }
        trade_df = pd.DataFrame(trade, index=[trade_datetime])
        if symbol in lib.list_symbols():
            lib.update(symbol, trade_df, upsert=True)
        else:
            lib.write(symbol, trade_df)

Function to start streaming all trades

In [ ]:
def stream_all_trades():
    client = ThetaClient(username="YOUR_USERNAME", passwd="YOUR_PASSWORD")

    client.connect_stream(callback)

    req_id = client.req_full_trade_stream_opt()

    response = client.verify(req_id)

    if client.verify(req_id) != StreamResponseType.SUBSCRIBED:
        raise Exception("Unable to stream.")

    time.sleep(120)  # Stream for 2 minutes

    print("Cancelling stream...")

    client.remove_full_trade_stream_opt()

Start streaming trades

In [ ]:
stream_all_trades()

Defragment symbols if necessary

In [ ]:
for symbol in lib.list_symbols():
    if lib.is_symbol_fragmented(symbol):
        lib.defragment_symbol_data(symbol)

List the symbols in the library

In [ ]:
lib.list_symbols()

Read all the data for one of the symbols (e.g., QQQ)

In [ ]:
qqq = lib.read("QQQ").data
display(qqq)

Use the query builder to find options with tight spread

In [ ]:
q = adb.QueryBuilder()
filter = (q.ask_price - q.bid_price) < 0.05
q = q[filter]
data = lib.read("QQQ", query_builder=q).data

In [ ]:
display(data)

Use the query builder to find options with more than 1 day to expiration

In [ ]:
q = adb.QueryBuilder()
filter = q.days_to_expiration > 1
q = q[filter].groupby("expiration").agg({"bid_size": "sum", "ask_size": "sum"})
data = lib.read("QQQ", query_builder=q).data.sort_index()

In [ ]:
display(data)

**Jason Strimpel** is the founder of <a href='https://pyquantnews.com/'>PyQuant News</a> and co-founder of <a href='https://www.tradeblotter.io/'>Trade Blotter</a>. His career in algorithmic trading spans 20+ years. He previously traded for a Chicago-based hedge fund, was a risk manager at JPMorgan, and managed production risk technology for an energy derivatives trading firm in London. In Singapore, he served as APAC CIO for an agricultural trading firm and built the data science team for a global metals trading firm. Jason holds degrees in Finance and Economics and a Master's in Quantitative Finance from the Illinois Institute of Technology. His career spans America, Europe, and Asia. He shares his expertise through the <a href='https://pyquantnews.com/subscribe-to-the-pyquant-newsletter/'>PyQuant Newsletter</a>, social media, and has taught over 1,000+ algorithmic trading with Python in his popular course **<a href='https://gettingstartedwithpythonforquantfinance.com/'>Getting Started With Python for Quant Finance</a>**. All code is for educational purposes only. Nothing provided here is financial advise. Use at your own risk.